# Ejemplo de simulación

Del Laboratorio anterior, dejamos creados nuestros modelos

In [1]:
import geopandas as gpd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.results as res
from biogeme.expressions import Beta, DefineVariable
import random as rd

```python
logprob = models.loglogit(V, av, choice)
bio.WEIGHT = 'Sup'
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "choice"
results = biogeme.estimate()
```

## Obteniendo los betas

Pueden directamente obtenerlos de la variable `results` con `results.getBetaValues()`. En este caso vamos a cargar directamente los archivos generados cuando ejecuten el Laboratorio anterior.

In [2]:
results_bid = res.bioResults(pickleFile = 'Ejemplo_bid.pickle')
results_choice = res.bioResults(pickleFile = 'choice.pickle')

In [3]:
results_bid.getBetaValues()

{'B_acc_op_cam_H2': 0.0018897931812969966,
 'B_acc_op_cam_H3': 0.0028537011909676377,
 'B_acc_op_veh_H2': -0.015682441051976963,
 'B_acc_op_veh_H3': -0.01725222279796192,
 'B_utilidad_H2': 56.18815976742222,
 'B_utilidad_H3': 58.735445957837996}

In [7]:
betas_choice = results_choice.getBetaValues()

## Ciudad en t=0 (2020)

In [6]:
ciudad_base = gpd.read_file('ciudad_sintetica.shp')
ciudad_base['precio_m2'] = ciudad_base.apply(lambda row: 10**5*rd.expovariate(row['acc_op_cam'] + row['acc_op_veh']/(ciudad_base['acc_op_cam'].sum()+ciudad_base['acc_op_veh'].sum())), axis = 1)
ciudad_base.head()

,cell,area,m2_op,acc_op_cam,acc_op_veh,geometry,precio_m2
0,1,25980.762114,1.496370,21.999698,3261.594956,"POLYGON ((0.000 1000.000, 100.000 1000.000, 15...",11334.423647
1,2,25980.762114,0.293320,20.939344,3283.642413,"POLYGON ((0.000 826.795, 100.000 826.795, 150....",2694.915215
2,3,25980.762114,3.416329,25.762190,3294.544635,"POLYGON ((0.000 653.590, 100.000 653.590, 150....",8204.780326
3,4,25980.762114,16.222386,47.757336,3293.653747,"POLYGON ((0.000 480.385, 100.000 480.385, 150....",1754.156902
4,5,25980.762114,8.038769,51.997028,3280.187080,"POLYGON ((0.000 307.180, 100.000 307.180, 150....",4745.553213


## Funciones para determinar las utilidades/disposiciones al pago

In [9]:
def utility(row, betas, variables, sup): #variables = ['acc_op_cam', 'acc_op_veh', 'precio_m2']
    ut = 0
    for v in variables:
        ut += row[v]*betas[f'B_{v}']
    ut += sup*betas['B_Sup']
    return ut

var = ['acc_op_cam', 'acc_op_veh', 'precio_m2']
sup = 50
ciudad_base['U_H1'] = ciudad_base.apply(lambda row: utility(row, betas_choice, var, sup), axis = 1)

In [10]:
ciudad_base.head()

,cell,area,m2_op,acc_op_cam,acc_op_veh,geometry,precio_m2,U_H1
0,1,25980.762114,1.496370,21.999698,3261.594956,"POLYGON ((0.000 1000.000, 100.000 1000.000, 15...",11334.423647,-3.271379
1,2,25980.762114,0.293320,20.939344,3283.642413,"POLYGON ((0.000 826.795, 100.000 826.795, 150....",2694.915215,-2.855424
2,3,25980.762114,3.416329,25.762190,3294.544635,"POLYGON ((0.000 653.590, 100.000 653.590, 150....",8204.780326,-3.139988
3,4,25980.762114,16.222386,47.757336,3293.653747,"POLYGON ((0.000 480.385, 100.000 480.385, 150....",1754.156902,-2.807065
4,5,25980.762114,8.038769,51.997028,3280.187080,"POLYGON ((0.000 307.180, 100.000 307.180, 150....",4745.553213,-2.944895
